In [1]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import numpy as np
from IPython.display import Markdown, display

/Users/joyce/.pyenv/versions/medgemma-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from huggingface_hub import login
# login(token="<>hugging_face_token</>")  # Uncomment this line if you need to log in using Hugging Face CLI

# LOGGED IN USING HUNNGINGFACE CLI

In [2]:
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.96it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use mps:0


In [3]:
medical_transcript = """
Physician: Good morning, Ms. Johnson. What brings you in today?

Patient: Morning, doctor. I’ve been having this persistent cough for the last two weeks. It started out dry, but now there’s some phlegm.

Physician: I see. Any fever or chills?

Patient: Yes, I had a mild fever three days ago, around 100.5°F, but it went away on its own.

Physician: Any shortness of breath or chest pain?

Patient: Not really chest pain, but my chest feels a bit tight, especially when I’m coughing a lot.

Physician: How about your energy levels or appetite?

Patient: I’ve been more tired than usual, and I haven’t had much of an appetite.

Physician: Any recent travel or contact with anyone who’s been sick?

Patient: No travel, but my son had a cold last week.

Physician: Okay. Let’s take a listen to your lungs.

[Stethoscope exam sounds]

Physician: I’m hearing some mild crackles in your lower lungs. No wheezing, which is good. I’ll check your oxygen level—98%, that’s normal.

Physician: I’d like to order a chest X-ray to rule out pneumonia, and we’ll also do a CBC and test for respiratory viruses. In the meantime, I’ll prescribe an inhaler to help with the chest tightness, and a cough suppressant so you can rest.

Patient: Sounds good. Should I stay home from work?

Physician: Yes, for at least a couple of days until the results come in and you’re feeling better. Stay hydrated, and rest as much as you can.

Patient: Will do. Thanks, doctor.

Physician: You’re welcome. We’ll follow up as soon as we have the test results.
"""

In [4]:
system_prompt = """
You are a clinical documentation assistant. Your task is to read medical transcripts 
(dialogues between clinicians and patients) and convert them into structured clinical notes using the SOAP format.

Follow these rules:

*S – Subjective*:
Include all information reported by the patient: symptoms, duration, history, complaints, and any relevant lifestyle or 
exposure context. Use the patient’s own words when possible (paraphrased for clarity).

*O – Objective*:
Include observable findings such as vital signs, physical exam results, lab tests, imaging results, and clinician 
observations during the encounter.

*A – Assessment*:
Provide a brief summary of the clinician’s diagnostic impression. Include possible or confirmed diagnoses.

*P – Plan*:
Outline the next steps recommended by the clinician. This can include prescriptions, tests to be ordered, 
referrals, follow-up instructions, and lifestyle recommendations.

Keep the format clear and professional. Do not include any parts of the transcript that are irrelevant or 
non-clinical. Do not invent information not found in the transcript. 
Allways use a bullet point format for each section of the SOAP note.

A comprehensive SOAP note has to take into account all subjective and objective information, 
and accurately assess it to create the patient-specific assessment and plan.

Here is the transcript from a medical record file from which you will be asked to extract relevant SOAP information:



"""

In [5]:
dummy_image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))


In [6]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": medical_transcript},
             {"type": "image", "image": dummy_image}
            
        ]
    }
]

In [7]:
output = pipe(text=messages, max_new_tokens=512)


In [8]:
display(Markdown(output[0]["generated_text"][-1]["content"]))


Here's a SOAP note based on the provided transcript:

**S (Subjective)**

*   Patient reports a persistent cough for two weeks, initially dry, now with some phlegm.
*   Patient experienced a mild fever (100.5°F) three days ago, which resolved on its own.
*   Patient reports chest tightness, especially with coughing.
*   Patient reports decreased energy levels and appetite.
*   Patient denies recent travel.
*   Patient's son had a cold last week.

**O (Objective)**

*   Lung exam reveals mild crackles in the lower lungs.
*   No wheezing noted.
*   Oxygen saturation: 98% (normal).

**A (Assessment)**

*   Possible bronchitis or other respiratory infection. Pneumonia is being ruled out with a chest X-ray.

**P (Plan)**

*   Order chest X-ray.
*   Order CBC (complete blood count).
*   Order respiratory viral panel.
*   Prescribe an inhaler for chest tightness.
*   Prescribe a cough suppressant.
*   Advise patient to stay home from work for at least two days.
*   Recommend hydration and rest.
*   Follow up with results of tests and further management as needed.


In [10]:
image_path = "medical_transcript.png"
image = Image.open(image_path)

In [11]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": ""},
            {"type": "image", "image": image},
        ]
    }
]

In [12]:
output = pipe(text=messages, max_new_tokens=512)


In [14]:
display(Markdown(output[0]["generated_text"][-1]["content"]))


Here's a SOAP note based on the provided transcript:

**S – Subjective:**

*   Patient reports abdominal pain for the past few days, described as cramping and comes and goes.
*   Pain started about four days ago, right after eating takeout food.
*   Patient experiences some nausea, but no vomiting or diarrhea.
*   Patient reports loose stools twice a day for the past couple of days.
*   Patient denies fever or chills.
*   Patient denies blood in stool or black, tarry stools.
*   Patient reports being unable to eat normally.
*   Patient states they have been eating a little less just because they feel full quickly and the pain gets worse after meals.

**O – Objective:**

*   Physical exam reveals some tenderness in the lower left quadrant.
*   Guarding is present but not guarding or rebound.
*   Bowel sounds are normal.

**A – Assessment:**

*   Possible mild case of gastroenteritis, but the clinician wants to rule out infection.

**P – Plan:**

*   Order a stool test to rule out infection and a basic metabolic panel.
*   Patient is instructed to stay hydrated, eat bland foods, and wait for the test results.
*   The clinician does not want to prescribe antibiotics unless something bacterial is found.
*   Patient is advised to call if the pain worsens or if they develop a fever.
*   The clinician will follow up in two weeks.
